In [ ]:
import arxiv
import openai
import requests
import base64
import time
import hashlib
import json
from datetime import datetime, timedelta, timezone
from IPython.display import display, Markdown, Image
from requests.exceptions import RequestException
from io import BytesIO

# Import secrets from separate file (not committed to git)
from secrets_config import (
    OPENAI_API_KEY,
    THREADS_USER_ID,
    THREADS_ACCESS_TOKEN,
    APP_SECRET,
    GITHUB_TOKEN,
    GITHUB_REPO,
    GITHUB_PAGES_SITE,
    FACEBOOK_ID,
    INSTAGRAM_ACCESS_TOKEN
)

# ============================================================================
# BLOG AUTOMATION CONFIGURATION - Modify these as needed
# ============================================================================

# arXiv paper settings
ARXIV_CATEGORIES = ["cs.AI", "cs.LG", "cs.CL", "cs.CV", "stat.ML"]
MAX_PAPERS_TO_PROCESS = 1       # Number of papers to fetch and potentially process
DAYS_BACK_TO_SEARCH = 7         # How many days back to search for new papers

# Content generation settings
BLOG_POST_LENGTH = "300-400"    # Target word count for blog posts
BLOG_POST_MAX_TOKENS = 500      # Max tokens for GPT response
BLOG_POST_TEMPERATURE = 0.7     # Creativity level (0.0-1.0)

# Threads post settings
THREADS_MAX_CHARS = 350         # Max characters for main Threads text
THREADS_HASHTAGS = "#AI #ArtificialIntelligence #MachineLearning #DataScience #Latest #Research #Arxiv #OpenAI"
THREADS_WAIT_TIME = 30          # Seconds to wait before publishing
THREADS_MAX_RETRIES = 3

# Image generation settings (DALL-E 3)
IMAGE_MODEL = "dall-e-3"        # "dall-e-2" or "dall-e-3"
IMAGE_QUALITY = "standard"      # "standard" or "hd" (hd costs more)
IMAGE_STYLE = "natural"         # "natural" or "vivid"
IMAGE_SIZE = "1024x1024"        # "1024x1024", "1792x1024", or "1024x1792"

# Processing settings
SKIP_EXISTING_POSTS = True      # Skip papers that already have blog posts
SAVE_IMAGES_TO_GITHUB = True    # Download and save images to prevent expiration
LINK_PREVIEW_WAIT = 30          # Seconds to wait for link preview generation
GITHUB_PAGES_IMAGE_WAIT = 60    # Seconds to wait for image deployment

# Testing/debugging settings
TEST_MODE = False               # Set to True to process only 1 paper for testing
VERBOSE_OUTPUT = True           # Show detailed processing information

# ============================================================================
# Auto-adjust settings based on test mode
# ============================================================================
if TEST_MODE:
    MAX_PAPERS_TO_PROCESS = 1
    print("🧪 TEST MODE ENABLED - Processing only 1 paper")

# Initialize the OpenAI client
client = openai.OpenAI(api_key=OPENAI_API_KEY)

# ============================================================================
# FUNCTIONS
# ============================================================================

def generate_harvard_reference(paper):
    authors = paper.authors
    
    if len(authors) == 1:
        author_str = authors[0].name
    elif len(authors) == 2:
        author_str = f"{authors[0].name} and {authors[1].name}"
    else:
        author_str = f"{authors[0].name} et al."
    
    year = paper.published.year
    title = paper.title
    
    reference = f"{author_str} ({year}) '{title}', arXiv preprint arXiv:{paper.get_short_id()}."
    
    return reference

def fetch_latest_papers(categories=None, max_results=None, days_back=None):
    # Use config defaults if no parameters provided
    if categories is None:
        categories = ARXIV_CATEGORIES
    if max_results is None:
        max_results = MAX_PAPERS_TO_PROCESS
    if days_back is None:
        days_back = DAYS_BACK_TO_SEARCH
        
    client_arxiv = arxiv.Client()  # Renamed to avoid confusion with OpenAI client
    cutoff_date = datetime.now(timezone.utc) - timedelta(days=days_back)
    
    category_query = " OR ".join([f"cat:{cat}" for cat in categories])
    
    search = arxiv.Search(
        query = f"({category_query})",
        max_results = max_results,
        sort_by = arxiv.SortCriterion.SubmittedDate
    )
    
    results = list(client_arxiv.results(search))
    recent_papers = [paper for paper in results if paper.published.replace(tzinfo=timezone.utc) > cutoff_date]
    
    if VERBOSE_OUTPUT:
        print(f"📊 Fetched {len(results)} papers, {len(recent_papers)} from last {days_back} days")
    
    return recent_papers

def generate_blog_post(paper):
    authors = ', '.join([author.name for author in paper.authors])
    prompt = f"""Write an engaging blog post about the following scientific paper:

Title: {paper.title}
Authors: {authors}
Abstract: {paper.summary}

The blog post should:
1. Explain the main findings in simple terms
2. Discuss potential real-world implications
3. Be engaging and accessible to a general audience
4. Be around {BLOG_POST_LENGTH} words long

Blog Post:"""

    try:
        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": "You are a helpful assistant that writes engaging blog posts about scientific papers."},
                {"role": "user", "content": prompt}
            ],
            max_tokens=BLOG_POST_MAX_TOKENS,
            temperature=BLOG_POST_TEMPERATURE
        )
        return response.choices[0].message.content.strip()
    except Exception as e:
        print(f"Error generating blog post: {e}")
        return None

def generate_threads_post(paper, blog_post_url):
    prompt = f"""Create a short, engaging post for Threads (max {THREADS_MAX_CHARS} characters) about this scientific paper:
    Title: {paper.title}
    
    Include a brief highlight of the research and its potential impact. 
    Do not include any hashtags or 'Read more' statements.
    """
    try:
        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": "You are a helpful assistant that creates engaging social media posts about scientific papers."},
                {"role": "user", "content": prompt}
            ],
            max_tokens=100,
            temperature=BLOG_POST_TEMPERATURE
        )
        threads_text = response.choices[0].message.content.strip().replace(":", "")
        
        full_post = f"{threads_text}\n\n{THREADS_HASHTAGS}\n\nRead more: {blog_post_url}"
        
        if len(full_post) > 500:
            available_chars = 500 - len(THREADS_HASHTAGS) - len(blog_post_url) - 15
            truncated_text = threads_text[:available_chars-3] + "..."
            full_post = f"{truncated_text}\n\n{THREADS_HASHTAGS}\n\nRead more: {blog_post_url}"
        
        return full_post
    except Exception as e:
        print(f"Error generating Threads post: {e}")
        return None
   
def generate_ai_image(paper, threads_post):
    """
    Generate an AI image using DALL-E 3 based on the research paper
    """
    # Extract key concepts from both title and abstract for better understanding
    title_words = paper.title.lower()
    abstract_words = paper.summary.lower()
    combined_text = f"{title_words} {abstract_words}"
    
    # Create a more detailed prompt based on the actual research content
    # Look for specific technical terms and concepts
    
    visual_elements = []
    domain_context = ""

    domain_color_palettes = {
        "computer vision and image processing": "vibrant blues, greens, and digital highlights",
        "natural language processing and text analysis": "warm oranges, soft whites, and typographic elements",
        "machine learning and neural networks": "deep purples, electric blues, and gradient flows",
        "robotics and autonomous systems": "industrial grays, precise reds, and mechanical details",
        "data science and analytics": "analytical blues, data-point reds, and statistical patterns",
        "medical AI and healthcare technology": "medical whites, diagnostic blues, and gentle healing greens",
        "quantum computing and quantum information": "quantum blues, probability purples, and uncertainty patterns",
        "reinforcement learning and intelligent agents": "reward golds, strategic blues, and environmental greens",
        "artificial intelligence research": "technological blues, innovative purples, and conceptual gradients"
    }
    
    # Computer Vision / Image Processing
    if any(word in combined_text for word in ['image', 'vision', 'visual', 'detection', 'segmentation', 'object', 'face', 'recognition']):
        domain_context = "computer vision and image processing"
        visual_elements.extend([
            "digital image grids with highlighted features",
            "geometric detection boxes and annotations",
            "layered visual processing pipelines",
            "camera or sensor imagery with analytical overlays"
        ])
    
    # Natural Language Processing
    elif any(word in combined_text for word in ['language', 'text', 'nlp', 'translation', 'sentiment', 'dialogue', 'conversation', 'llm']):
        domain_context = "natural language processing and text analysis"
        visual_elements.extend([
            "flowing text streams transforming between languages",
            "word clouds with connecting semantic relationships",
            "chat bubbles and conversation interfaces",
            "linguistic trees and grammar structures"
        ])
    
    # Machine Learning / Neural Networks
    elif any(word in combined_text for word in ['neural', 'network', 'learning', 'training', 'model', 'algorithm', 'optimization']):
        domain_context = "machine learning and neural networks"
        visual_elements.extend([
            "interconnected neural network nodes with flowing data",
            "gradient flows and optimization landscapes",
            "training data points clustering and separating",
            "layered network architectures with information flow"
        ])
    
    # Robotics / Autonomous Systems
    elif any(word in combined_text for word in ['robot', 'autonomous', 'control', 'manipulation', 'navigation', 'motion']):
        domain_context = "robotics and autonomous systems"
        visual_elements.extend([
            "robotic arms with precise movement trajectories",
            "autonomous vehicles navigating environments",
            "sensor data visualization around robotic systems",
            "mechanical components with motion indicators"
        ])
    
    # Data Science / Analysis
    elif any(word in combined_text for word in ['data', 'analysis', 'mining', 'clustering', 'classification', 'prediction', 'statistics']):
        domain_context = "data science and analytics"
        visual_elements.extend([
            "data visualization charts and graphs",
            "clustering patterns and data point relationships",
            "statistical distributions and trend lines",
            "database connections and information flow diagrams"
        ])
    
    # Healthcare / Medical AI
    elif any(word in combined_text for word in ['medical', 'health', 'diagnosis', 'patient', 'clinical', 'drug', 'disease']):
        domain_context = "medical AI and healthcare technology"
        visual_elements.extend([
            "medical scan imagery with AI analysis highlights",
            "molecular structures and drug interactions",
            "patient data flows and diagnostic pathways",
            "healthcare monitoring interfaces and vital signs"
        ])
    
    # Quantum Computing
    elif any(word in combined_text for word in ['quantum', 'qubit', 'entanglement', 'superposition']):
        domain_context = "quantum computing and quantum information"
        visual_elements.extend([
            "quantum state visualizations with wave functions",
            "entangled particle representations",
            "quantum circuit diagrams with gate operations",
            "probabilistic quantum measurement outcomes"
        ])
    
    # Reinforcement Learning / Gaming
    elif any(word in combined_text for word in ['reinforcement', 'reward', 'policy', 'agent', 'environment', 'game']):
        domain_context = "reinforcement learning and intelligent agents"
        visual_elements.extend([
            "agent-environment interaction loops",
            "reward signal visualizations and policy maps",
            "decision trees and action space exploration",
            "learning progress and performance curves"
        ])
    
    # Default AI Research
    else:
        domain_context = "artificial intelligence research"
        visual_elements.extend([
            "abstract AI concept representations",
            "algorithmic flow diagrams",
            "digital transformation processes",
            "computational thinking visualizations"
        ])
    
    # Extract specific paper concepts to make it even more targeted
    specific_concepts = []
    
    # Look for specific techniques or models mentioned
    techniques = ['transformer', 'cnn', 'rnn', 'lstm', 'bert', 'gpt', 'diffusion', 'gan', 'vae', 'attention']
    for tech in techniques:
        if tech in combined_text:
            specific_concepts.append(tech)
    
    # Look for application domains
    applications = ['autonomous driving', 'medical imaging', 'speech recognition', 'recommendation', 'translation']
    for app in applications:
        if app.replace(' ', '') in combined_text.replace(' ', ''):
            specific_concepts.append(app)
    
    # Create the enhanced prompt
    base_elements = ", ".join(visual_elements[:2])  # Use first 2 visual elements
    
    concept_addition = ""
    if specific_concepts:
        concept_addition = f", specifically highlighting {specific_concepts[0]} concepts"
    
    # Extract a key insight from the title for visual focus
    title_focus = ""
    if any(word in title_words for word in ['novel', 'new', 'improved', 'efficient', 'robust']):
        title_focus = " showcasing innovation and advancement"
    elif any(word in title_words for word in ['multi', 'cross', 'joint', 'unified']):
        title_focus = " emphasizing integration and connectivity"
    elif any(word in title_words for word in ['real-time', 'fast', 'rapid', 'efficient']):
        title_focus = " conveying speed and efficiency"
    
    # Get the domain-specific color palette
    color_palette = domain_color_palettes.get(domain_context, "modern professional colors with subtle gradients")
    
    prompt = f"""
    Create a modern, sophisticated illustration representing {domain_context}. 
    The image should feature {base_elements}{concept_addition}{title_focus}.
    
    Use a palette of {color_palette}. 
    The style should be clean, minimalist, and technically accurate, suitable for 
    a research publication or technical blog. 
    
    Avoid any text, human figures, or company logos. Show concrete, recognizable 
    technical elements and concepts that directly relate to {domain_context.split(' ')[0]} 
    research. Make the connection to the research topic immediately clear and visually 
    representative of the actual work being done.
    """
    
    try:
        print(f"Generating DALL-E 3 image for: {paper.title[:50]}...")
        print(f"🎨 Domain: {domain_context}")
        if specific_concepts:
            print(f"🔍 Concepts: {', '.join(specific_concepts[:2])}")
        
        response = client.images.generate(
            model=IMAGE_MODEL,
            prompt=prompt,
            n=1,
            size=IMAGE_SIZE,
            quality=IMAGE_QUALITY,
            style=IMAGE_STYLE
        )
        
        image_url = response.data[0].url
        print("✅ DALL-E 3 image generated successfully!")
        
        # Display the image in the notebook (if running in Jupyter)
        try:
            from IPython.display import display, Image
            display(Image(url=image_url))
        except ImportError:
            print(f"Image URL: {image_url}")
        
        return image_url
        
    except Exception as e:
        print(f"❌ Error generating DALL-E 3 image: {e}")
        
        # Fallback: try with a simpler but still specific prompt
        try:
            print("Trying with simplified prompt...")
            simple_prompt = f"A clean, modern illustration of {domain_context}, minimalist style, blue and purple gradient, technical diagram aesthetic"
            
            response = client.images.generate(
                model=IMAGE_MODEL,
                prompt=simple_prompt,
                n=1,
                size=IMAGE_SIZE,
                quality=IMAGE_QUALITY
            )
            
            image_url = response.data[0].url
            print("✅ Fallback image generated successfully!")
            return image_url
            
        except Exception as e2:
            print(f"❌ Fallback also failed: {e2}")
            return None

def download_and_save_image(image_url, paper_short_id, date):
    """
    Download the AI image and save it to GitHub repository
    This prevents the image from expiring
    """
    import requests
    import base64
    
    try:
        # Download the image
        print("Downloading image...")
        img_response = requests.get(image_url)
        img_response.raise_for_status()
        
        # Create filename
        image_filename = f"assets/images/{date}-{paper_short_id}.png"
        
        # Encode image for GitHub API
        encoded_image = base64.b64encode(img_response.content).decode("utf-8")
        
        # Upload to GitHub
        url = f"https://api.github.com/repos/{GITHUB_REPO}/contents/{image_filename}"
        headers = {
            "Authorization": f"token {GITHUB_TOKEN}",
            "Accept": "application/vnd.github+json",
            "X-GitHub-Api-Version": "2022-11-28"  # Use the API version from docs
        }
        
        # Always try to create as new file first (no sha parameter)
        print(f"📤 Creating new file: {image_filename}")
        data = {
            "message": f"Add image for blog post {paper_short_id}",
            "content": encoded_image
        }
        
        response = requests.put(url, headers=headers, json=data)
        print(f"📥 Upload response: {response.status_code}")
        
        if response.status_code == 201:
            # Successfully created new file
            github_image_url = f"https://{GITHUB_PAGES_SITE}/{image_filename}"
            print(f"✅ Image saved to GitHub: {github_image_url}")
            
            # Wait for GitHub Pages to deploy the image
            print(f"⏳ Waiting for GitHub Pages to deploy image ({GITHUB_PAGES_IMAGE_WAIT} seconds)...")
            time.sleep(GITHUB_PAGES_IMAGE_WAIT)
            
            # Test if the image is accessible
            try:
                test_response = requests.head(github_image_url, timeout=10)
                if test_response.status_code == 200:
                    print("✅ GitHub Pages image is accessible!")
                    return github_image_url
                else:
                    print(f"⚠️ GitHub Pages image not ready (status: {test_response.status_code})")
                    print("📎 Using original OpenAI URL as fallback")
                    return image_url
            except Exception as e:
                print(f"⚠️ Cannot verify GitHub Pages image accessibility: {e}")
                print("📎 Using original OpenAI URL as fallback")
                return image_url
                
        elif response.status_code == 422:
            # File might already exist, try to update it
            print("🔄 File might exist, checking and updating...")
            
            # Get the existing file info
            check_response = requests.get(url, headers=headers)
            if check_response.status_code == 200:
                file_info = check_response.json()
                print(f"📄 File exists, updating with sha: {file_info['sha'][:8]}...")
                
                # Update with sha
                update_data = {
                    "message": f"Update image for blog post {paper_short_id}",
                    "content": encoded_image,
                    "sha": file_info["sha"]
                }
                
                update_response = requests.put(url, headers=headers, json=update_data)
                if update_response.status_code == 200:
                    github_image_url = f"https://{GITHUB_PAGES_SITE}/{image_filename}"
                    print(f"✅ Image updated on GitHub: {github_image_url}")
                    return github_image_url
                else:
                    print(f"❌ Failed to update: {update_response.status_code}")
                    print(f"Response: {update_response.text}")
                    return image_url
            else:
                print(f"❌ Cannot check file existence: {check_response.status_code}")
                print("📎 Using original OpenAI URL as fallback")
                return image_url
        else:
            print(f"❌ Failed to save image to GitHub: {response.status_code}")
            print(f"Response: {response.text}")
            print("📎 Using original OpenAI URL as fallback")
            return image_url
            
    except Exception as e:
        print(f"❌ Error saving image: {e}")
        print("📎 Using original OpenAI URL as fallback")
        return image_url

def create_github_blog_post(paper, content, date, short_id, image_url):
    # Use consistent short_id for the file name
    file_name = f"{date}-{short_id}.md"
    
    # Generate Harvard reference
    harvard_reference = generate_harvard_reference(paper)
    
    # Fixed: No indentation in front matter
    file_content = f"""---
layout: post
title: "{paper.title}"
date: {date} {datetime.now().strftime('%H:%M:%S +0000')}
categories: [blog, AI, research]
image: {image_url}
---
![AI Generated Image]({image_url})

{content}

## Original Research Paper
For more details, please refer to the original research paper:
[{paper.title}]({paper.entry_id})

## Reference
{harvard_reference}
"""
    
    encoded_content = base64.b64encode(file_content.encode("utf-8")).decode("utf-8")
    url = f"https://api.github.com/repos/{GITHUB_REPO}/contents/_posts/{file_name}"
    headers = {
        "Authorization": f"token {GITHUB_TOKEN}",
        "Accept": "application/vnd.github.v3+json"
    }
    
    # Check if file already exists
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        print(f"Blog post already exists: {file_name}")
        return False, ""

    # File doesn't exist, create new file
    data = {
        "message": f"Add new blog post: {paper.title}",
        "content": encoded_content
    }

    response = requests.put(url, headers=headers, json=data)
    if response.status_code != 201:
        print(f"GitHub API Error: {response.status_code}")
        print(f"Response content: {response.text}")
        return False, ""
    
    # Construct the URL based on the file name
    post_url = f"https://{GITHUB_PAGES_SITE}/{date.replace('-', '/')}/{short_id}/"
    return True, post_url
   

def check_existing_post(short_id, date):
    file_name = f"{date}-{short_id}.md"
    url = f"https://api.github.com/repos/{GITHUB_REPO}/contents/_posts/{file_name}"
    headers = {
        "Authorization": f"token {GITHUB_TOKEN}",
        "Accept": "application/vnd.github.v3+json"
    }
    response = requests.get(url, headers=headers)
    return response.status_code == 200

def create_media_container(access_token, user_id, text, image_url):
    url = f"https://graph.threads.net/v1.0/{user_id}/threads"
    
    params = {
        "media_type": "IMAGE",
        "image_url": image_url,
        "text": text,
        "access_token": access_token
    }
    
    try:
        response = requests.post(url, params=params)
        response.raise_for_status()
        print(f"Create Media Container Status Code: {response.status_code}")
        return response.json()
    except requests.RequestException as e:
        print(f"Error creating media container: {e}")
        if hasattr(e, 'response') and e.response is not None:
            print(f"Response content: {e.response.text}")
        return None

def publish_thread(access_token, user_id, creation_id):
    url = f"https://graph.threads.net/v1.0/{user_id}/threads_publish"
    
    params = {
        "creation_id": creation_id,
        "access_token": access_token
    }
    
    try:
        response = requests.post(url, params=params)
        response.raise_for_status()
        print(f"Publish Thread Status Code: {response.status_code}")
        return response.json()
    except requests.RequestException as e:
        print(f"Error publishing thread: {e}")
        if hasattr(e, 'response') and e.response is not None:
            print(f"Response content: {e.response.text}")
        return None

def post_to_threads(text, image_url, access_token, user_id, initial_wait=30, max_retries=3):
    for attempt in range(max_retries):
        try:
            # Step 1: Create media container
            container = create_media_container(access_token, user_id, text, image_url)
            if container is None or 'id' not in container:
                print("Failed to create media container.")
                return False

            container_id = container['id']
            print(f"Media container created with ID: {container_id}")

            # Wait before publishing
            print(f"Waiting {initial_wait} seconds before publishing...")
            time.sleep(initial_wait)

            # Step 2: Publish the thread
            publish_result = publish_thread(access_token, user_id, container_id)
            if publish_result is None or 'id' not in publish_result:
                print("Failed to publish thread.")
                return False

            print(f"Successfully posted to Threads with ID: {publish_result['id']}")
            return True

        except Exception as e:
            print(f"Error posting to Threads: {e}")
            
            if attempt < max_retries - 1:
                print(f"Retrying in {initial_wait} seconds...")
                time.sleep(initial_wait)
            else:
                print("Max retries reached. Failed to post to Threads.")
                return False

    return False

def main():
    try:
        if VERBOSE_OUTPUT:
            print("🚀 Starting automated blog creation process...")
            print(f"📝 Configuration: {MAX_PAPERS_TO_PROCESS} papers, {DAYS_BACK_TO_SEARCH} days back")
            print(f"🔍 Categories: {', '.join(ARXIV_CATEGORIES)}")
        
        papers = fetch_latest_papers()
        if not papers:
            print(f"No recent papers found in categories: {ARXIV_CATEGORIES}")
            return

        processed_count = 0
        for paper in papers:
            display(Markdown(f"## Processing: {paper.title}"))
            
            short_id = hashlib.md5(paper.title.encode()).hexdigest()[:8]
            
            date = datetime.now().strftime("%Y-%m-%d")
            if SKIP_EXISTING_POSTS and check_existing_post(short_id, date):
                print(f"Blog post already exists for: {paper.title}")
                print("Skipping to next paper...")
                continue

            blog_post = generate_blog_post(paper)
            if not blog_post:
                print(f"Failed to generate blog post for: {paper.title}")
                print("Skipping to next paper...")
                continue

            display(Markdown(f"### Original Paper: [{paper.entry_id}]({paper.entry_id})"))
            display(Markdown(blog_post))
            
            # Generate a temporary post URL
            temp_post_url = f"https://{GITHUB_PAGES_SITE}/{date.replace('-', '/')}/{short_id}/"
            
            # Generate Threads post first
            threads_post = generate_threads_post(paper, temp_post_url)
            if not threads_post:
                print("Failed to generate Threads post. Skipping to next paper...")
                continue
            
            # Generate image based on paper content
            image_url = generate_ai_image(paper, threads_post)
            if not image_url:
                print("Failed to generate AI image. Skipping to next paper...")
                continue
            
            # Download and save the image to prevent expiration
            if SAVE_IMAGES_TO_GITHUB:
                image_url = download_and_save_image(image_url, short_id, date)
            
            # Create GitHub blog post with the generated image
            success, post_url = create_github_blog_post(paper, blog_post, date, short_id, image_url)
            
            if success:
                print(f"Successfully created blog post on GitHub: {post_url}")
                
                # Add delay to allow for link preview generation
                print(f"Waiting {LINK_PREVIEW_WAIT} seconds for link preview generation...")
                time.sleep(LINK_PREVIEW_WAIT)
                
                # Update the Threads post with the correct URL if it changed
                if post_url != temp_post_url:
                    threads_post = threads_post.replace(temp_post_url, post_url)
                
                display(Markdown(f"### Threads Post:\n{threads_post}"))
                if post_to_threads(threads_post, image_url, THREADS_ACCESS_TOKEN, THREADS_USER_ID, THREADS_WAIT_TIME, THREADS_MAX_RETRIES):
                    print("Successfully posted to Threads with image!")
                else:
                    print("Failed to post to Threads.")
                
                processed_count += 1
            else:
                print("Failed to create blog post on GitHub.")
        
        if VERBOSE_OUTPUT:
            print(f"\n✅ Processing complete! Created {processed_count} new blog posts.")

    except Exception as e:
        print(f"An error occurred: {e}")
        raise

if __name__ == "__main__":
    main()

🚀 Starting automated blog creation process...
📝 Configuration: 1 papers, 7 days back
🔍 Categories: cs.AI, cs.LG, cs.CL, cs.CV, stat.ML
📊 Fetched 1 papers, 1 from last 7 days


## Processing: How does Alignment Enhance LLMs' Multilingual Capabilities? A Language Neurons Perspective

### Original Paper: [http://arxiv.org/abs/2505.21505v1](http://arxiv.org/abs/2505.21505v1)

Are you fascinated by the inner workings of language processing in our brains? A new scientific paper titled "How does Alignment Enhance LLMs' Multilingual Capabilities? A Language Neurons Perspective" delves into the intricate mechanisms that underlie multilingual capabilities in Language Model Models (LLMs). Let's break down the key findings of this study in a way that is easy to understand.

The researchers explored how aligning high-resource languages with low-resource languages can boost LLMs' ability to understand and generate text in multiple languages. They discovered that within the brains of these models, there are specialized neurons that are activated when processing different languages. These language-specific neurons play a crucial role in enabling LLMs to effectively navigate multilingual scenarios.

By developing a sophisticated neuron identification algorithm, the researchers were able to categorize these neurons into language-specific, language-related, and language-agnostic types. This classification allowed them to dissect the internal processes involved in multilingual inference, which include multilingual understanding, shared semantic space reasoning, multilingual output space transformation, and vocabulary space outputting.

One fascinating observation from the study was the phenomenon of "Spontaneous Multilingual Alignment," where the models naturally adjust their alignment without explicit instructions. This natural ability to align languages could have significant implications for real-world applications, such as improving machine translation systems, language learning tools, and cross-cultural communication platforms.

Understanding how LLMs leverage different types of neurons to enhance their multilingual capabilities provides valuable insights for researchers looking to advance the field of natural language processing. By studying the neural mechanisms involved in language processing, we can unravel the complexities of multilingual communication and develop more sophisticated AI systems that bridge linguistic barriers.

In conclusion, this research sheds light on the intricate interplay between language-specific neurons and multilingual alignment in LLMs, offering a deeper understanding of how these models navigate diverse linguistic landscapes. The implications of this study extend beyond academia and into practical applications that could revolutionize how we interact with and understand languages in our increasingly globalized world.

Generating DALL-E 3 image for: How does Alignment Enhance LLMs' Multilingual Capa...
🎨 Domain: natural language processing and text analysis
✅ DALL-E 3 image generated successfully!


📤 Creating new file: assets/images/2025-05-28-95a4152d.png
📥 Upload response: 201
✅ Image saved to GitHub: https://porkpy.github.io/research_blogger/assets/images/2025-05-28-95a4152d.png
⏳ Waiting for GitHub Pages to deploy image (60 seconds)...
✅ GitHub Pages image is accessible!
Successfully created blog post on GitHub: https://porkpy.github.io/research_blogger/2025/05/28/95a4152d/
Waiting 30 seconds for link preview generation...


### Threads Post:
🧠🔗 Dive into the fascinating world of language neurons! This study explores how alignment boosts multilingual capabilities in LLMs. Discover the brain's role in language processing and its potential impact on language learning. #neuroscience #multilingualism

#AI #ArtificialIntelligence #MachineLearning #DataScience #Latest #Research #Arxiv #OpenAI

Read more: https://porkpy.github.io/research_blogger/2025/05/28/95a4152d/

Create Media Container Status Code: 200
Media container created with ID: 17867077875389393
Waiting 30 seconds before publishing...
Publish Thread Status Code: 200
Successfully posted to Threads with ID: 18063894143281698
Successfully posted to Threads with image!

✅ Processing complete! Created 1 new blog posts.
